<a href="https://colab.research.google.com/github/dastone16/python_package/blob/main/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

def mult3(number): #test for definition_package.
  return number*3


def data_description(data,label):
  description = data.describe().transpose()
  #print(description)
  description.to_csv(save_path + label + '.csv')

#Save but likely not necessary
def mfile_import(path,columns,date_name,dateformat="%d/%m/%Y %H:%M:%S"):
  #Date formating help: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
  filepaths = [f for f in listdir(path) if f.endswith('.csv')]
  output_data = pd.DataFrame()
  for file in filepaths:
    temp_data= pd.read_csv(path + file,header=0,usecols=columns,parse_dates=[date_name]) 
    temp_data[date_name]= pd.to_datetime(temp_data[date_name].dt.strftime(dateformat)) #needed to shift unique format
    output_data = pd.concat([output_data,temp_data])
   
  return output_data

def gun_corr(data,spraygun):
  Spray_NN = pd.pivot_table(data[data['Machine']==spraygun],columns='Variable',values='Value by Gun',index=date_label)
  Spray_NN.dropna()
  Spray_NN_Day = Spray_NN.resample('D').mean()
  Spray_NN_table = Spray_NN_Day.corr()
  Spray_NN_table.to_csv(save_path + spraygun + 'gun.correlation.csv')
  x_len,y_len=Spray_NN_table.shape

  f = plt.figure(figsize=(10, 10))
  plt.matshow(Spray_NN_table, fignum=f.number)
  plt.xticks(range(Spray_NN_table.shape[1]), Spray_NN_table.columns, fontsize=12, rotation=90)
  plt.yticks(range(Spray_NN_table.shape[1]), Spray_NN_table.columns, fontsize=12)
  cb = plt.clim(-1,1) #forced limits to -1,1.
  cb = plt.colorbar()
  for i in range(x_len):
    for j in range(y_len):
        text = plt.text(j, i, round(Spray_NN_table.iloc[i, j],2), ha='center', va='center', color='r')
  
  cb.ax.tick_params(labelsize=16)
  plt.title(spraygun, fontsize=18,horizontalalignment = 'center',y=1.20);

  f.savefig(save_path + spraygun + 'Correlation.plot.Daily.jpg')

#creates scatter plot 
def scatter_plot(data,plotx, ploty,Line):
  #plotx = 'Stage 2 Free Acid'
  #ploty = 'Spray-31 S Dist M' 
  plt.figure()
  #ax = fig.gca(autoscalex_on=True)
  plt.scatter(data[plotx],data[ploty])
  #plt.yscale('log')
  plt.ylabel(ploty)
  plt.xlabel(plotx)
  z = np.polyfit(data[plotx], data[ploty], 1)
  p = np.poly1d(z)
  #plt.plot(data[plotx],p(data[plotx]),"r--",)
  #plt.savefig(save_path + Line + ploty + ' vs.' + plotx + '.scatter.jpg')
  X = data[plotx]
  Y = data[ploty]

  X = sm.add_constant(X) # adding a constant

  model = sm.OLS(Y, X, missing='drop').fit()
  #predictions = model.predict(X) 

  print_model = model.summary()
  return print(print_model)

def correlation_plot(data,resample,output_name,size=(18,12),font=18,plot=True,all_factors=True,corr_variable='Metal Exposure (mA)'):
  data_corr = data.resample(resample).mean().corr()
  data_corr = data_corr.dropna(how='all')
  data_corr = data_corr.dropna(how='all',axis='columns')
  if plot:
    f = plt.figure(figsize=size)
    plt.matshow(data_corr, fignum=f.number)
    plt.xticks(range(data_corr.shape[1]), data_corr.columns, fontsize=font, rotation=90)
    plt.yticks(range(data_corr.shape[1]), data_corr.columns, fontsize=font)
    cb = plt.clim(-1,1) 
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    #plt.title('Correlation Matrix L12', fontsize=16);

    f.savefig(save_path + output_name + 'Correlation.plot.Daily.jpg')
  if all_factors:
    # correlation between all factors.
    data_corr_table =  data_corr.stack().reset_index()
    data_corr_table.columns = ['Factor1','Factor2',"Correlation"]
    data_corr_table.drop(data_corr_table[data_corr_table['Factor1']==data_corr_table['Factor2']].index,inplace=True)
    data_corr_table.head()

    data_corr_table = data_corr_table.iloc[(-np.abs(data_corr_table['Correlation'].values)).argsort()]
    data_corr_table = data_corr_table.drop_duplicates('Correlation')
    data_corr_table.to_csv(save_path + output_name + 'Full.correlation.Table.csv')
  if mA:
  #Defined correlation
    data_corr_ma = data_corr.loc[[corr_variable],:]
    data_corr_ma = data_corr_ma.transpose()
    #data_corr_ma
    data_mA_table = data_corr_ma.iloc[(-np.abs(data_corr_ma[corr_variable].values)).argsort()]
    data_mA_table.rename(columns={corr_variable:'Correlation to '+ corr_variable}, inplace=True)
    #data_mA_table.head()

    data_mA_table.to_csv(save_path + output_name + 'correlation.to.'+corr_variable+'.csv')

def date_compare(data,date_label,start_date,trans_date,fin_date,
                 output_name,resample='H',label1="Before",label2="After",corr="Metal Exposure (mA)",
                 raw=False):
  """
  Data must be dataframe indexed to date.
  resample is time of resampling typically daily 'D', rarely hourly 'H'
  data parsed from start to finish with transition date in middle
  corr: correlation to.  If none correlate to all.
  raw=False will not prettify output columns.
  """
  Summary=pd.DataFrame()
  
  #reindex by date
  data2 = data.resample(resample).mean() #resample by second to get full statistics 
  
  #section by date
  before = data2[start_date:trans_date].copy()
  after = data2[trans_date:fin_date].copy()
  
  correlation_plot(before,'D',output_name+label1,plot=False,all_factors=False)
  correlation_plot(after,'D',output_name+label2,plot=False,all_factors=False)
  before_T = before.transpose()
  #+ " " + str(trans_date)+
  Summary[label1 + 'Mean'] = np.round(before_T.mean(axis=1),4)
  Summary[label1 + ' Std1'] = before_T.std(axis=1)
  Summary[label1 + ' Count1'] = before_T.count(axis=1)
  Summary[label1 + 'StdError'] = sp.sem(before_T,nan_policy='omit', axis=1)
  after_T = after.transpose()
  
  Summary[label2 +' Mean'] = np.round(after_T.mean(axis=1),4)
  Summary[label2 + ' Std2'] = after_T.std(axis=1)
  Summary[label2 + ' Count2'] = after_T.count(axis=1)
  Summary[label2 + 'StdError'] = sp.sem(after_T,nan_policy='omit', axis=1)

  Summary['Difference']=np.round((Summary[label1 + ' Mean'] - Summary[label2 +' Mean']),4)
  Summary['Std Error Diff'] = np.sqrt((Summary[label1 +'StdError']**2) + (Summary[label2 +'StdError']**2))
  Summary['df']=Summary[label1 + ' Count1']+Summary[label2 + ' Count2']-2
  Summary['T-Stat'] = Summary['Difference']/Summary['Std Error Diff']
  Summary['P-value'] = np.round((1.0 - sp.t.cdf(abs(Summary['T-Stat']),Summary['df']))*2.0,3)
  #Summary[label1 + ' Correlation to mA']=before_D_corr.loc[['Metal Exposure (mA)'],:]
  #Summary[label2 + ' Correlation to mA']=after_D_corr.loc[['Metal Exposure (mA)'],:]
  Summary = Summary.iloc[(-np.abs(Summary['T-Stat'].values)).argsort()]
  if raw == False:
    Summary.drop(columns=[label1 + ' Std1',label1 + ' Count1',label1 + 'StdError',
                          label2 + ' Std2',label2 + ' Count2',label2 + 'StdError',
                          'Std Error Diff','df'], inplace=True)
    Summary.to_csv(save_path + output_name+label1+label2+'.diff.csv')
  else:
    Summary.to_csv(save_path + output_name+label1+label2+'.diff.csv')
    pass
  return Summary

  
  
def high_low_days(data,resample,Output_name,variable,days=10):
  #input already averaged data by day.  

  #originally used for only mA, switched to any variable, but temporary variables inside function still refer to mA.
  data_Day = data.resample(resample).mean()
  data_Day = data_Day.dropna(how='all')
  data_Day = data_Day.dropna(how='all',axis='columns')
  data_Day = data_Day.iloc[(-np.abs(data_Day[variable].values)).argsort()]
  data_High20_mA_Days = data_Day.iloc[0:days,:]
  data_Low20_mA_Days = data_Day.iloc[-days:-1,:]
  #labeled mA because originally only written for mA.  Added ability to input vs any variable.

  data_High_mean = data_High20_mA_Days.resample('10AS').mean() #resample by year to average
  data_High_mean = data_High_mean.reset_index(drop=True).transpose()
  data_Low_mean = data_Low20_mA_Days.resample('10AS').mean()
  data_Low_mean = data_Low_mean.reset_index(drop=True).transpose()
  data_Range_max = data_Day.resample('10AS').max()  # by year to get entire range
  data_Range_max = data_Range_max.reset_index(drop=True).transpose()
  data_Range_min = data_Day.resample('10AS').min()
  data_Range_min = data_Range_min.reset_index(drop=True).transpose()
  data_High_mean.rename(columns={0:'Avg: '+str(days)+' Highest Days'}, inplace=True)
  data_Low_mean.rename(columns={0:'Avg: '+str(days)+' Lowest Days'}, inplace=True)
  data_Range_max.rename(columns={0:'Max'}, inplace=True)
  data_Range_min.rename(columns={0:'Min'}, inplace=True)

  data_Summary = data_Low_mean
  data_Summary['Avg: '+str(days)+' Highest Days'] = data_High_mean['Avg: '+str(days)+' Highest Days']
  data_Summary['Difference'] = data_High_mean['Avg: '+str(days)+' Highest Days']-data_Low_mean['Avg: '+str(days)+' Lowest Days']
  data_Summary['Max'] = data_Range_max['Max']
  data_Summary['Min'] = data_Range_min['Min']
  data_Summary['Range'] = data_Summary['Max'] - data_Summary['Min']
  data_Summary['% of Range'] = data_Summary['Difference']/data_Summary['Range']*100
  data_Summary = data_Summary.iloc[(-np.abs(data_Summary['% of Range'].values)).argsort()]
  data_Summary.to_csv(save_path + Output_name + variable + str(days)+'days.High.Low.diff.csv')
  return data_Summary


#Elapsed color definitions
def Elapse_cell_color(val):
  if val < 22:
    color= 'rgb(146,208,80)'#green
  elif val <27:
    color = 'yellow'
  elif math.isnan(val):
    color = 'white' 
  else:
    color = 'rgb(225,45,45)'

  return 'background-color: %s' % color
def Elapse_text_color(val):
  if math.isnan(val):
    color = 'white'
  else:
    color = 'black'
  return 'color: %s' % color

def elapsed(data,date_start,group=['Plant','Line'],variable_name='Variable',variable="Metal Exposure (mA)",multilevel=True):
  """
  data=dataframe not pivot
  group = how to group data in output
  Returns: elapsed value by group,  
  """
  data_L1 = data[data[date_label]>date_start].copy() #greater means closer to today
  data_L1 = data_L1.sort_values(by=date_label)
  data_L1 = data_L1[data_L1[variable_name]==variable].copy() #variable_name makes this generic.
  data_L1_S = data_L1[data_L1['Sample No']==1].copy()#S= first measurement

  data_L1_S.sort_values(group,inplace=True,ascending=True)
  data_L1_S['Lagged_Time']=data_L1_S[date_label].shift(1)
  data_L1_S.dropna(inplace=True)
  data_L1_S['Elapsed Time'] = (data_L1_S[date_label] - data_L1_S['Lagged_Time']).astype("timedelta64[m]")
  #for multilevel
  if multilevel:
    elapsed_by = data_L1_S.groupby(group)['Elapsed Time'].median().unstack().sort_index(axis=1)
  else:
    elapsed_by = data_L1_S.groupby(group)['Elapsed Time'].median()
    elapsed_by = pd.DataFrame(elapsed_by)
  #elapsed_by = data_L1_S.groupby(group)['Elapsed Time'].median()
  #elapsed_by

  elapsed_by_s = elapsed_by.style.applymap(Elapse_cell_color)\
                  .format("{:3.1f}")\
                  .set_properties(**{'width':'100px', 'text-align':'center'})\
                  .applymap(Elapse_text_color)\
                  .set_table_styles([{'selector': 'th','props': [('background-color', 'rgb(255,242,204)'),('border-style','solid'),('border-width','1px'),('color', 'black')]}])
  elapsed_by_s.index.name = None
    
  return elapsed_by, elapsed_by_s

def subplotdata (data,yaxis,yaxislabels,limit,title,save_label,after_date=None):
  """
  #data is a list of data frames to plot
  #yaxis is a list of columns used for plotting
  #yaxislabels is a list of the y-axis in the plot
  #limit is boolean for limits adjusted for plots
  #title is title of all plot
  #save label is name of file to be saved.
  #after date
  """
  x_len=len(yaxis)
  y_len=len(data[0].groupby(level=0))
  colors = ['orange','blue','green','red','purple','gray','brown','pink']
  fig, axs = plt.subplots(x_len,y_len,sharey='row',sharex='col',figsize=(12,12))

  for j in range(x_len):
    axs[j,0].set_ylabel(yaxislabels[j])
    if limit[j]:
      axs[j,0].set_ylim(data[j][yaxis[j]].min()-(0.2*data[j][yaxis[j]].min()),data[j][yaxis[j]].max()+(0.1*data[j][yaxis[j]].max()))
    for i in range(y_len):
      data[j].loc[[lines[i]],[yaxis[j]]].plot(kind='bar', ax=axs[j,i],legend=None,color=colors[i])
  axs[0,y_len//2].set_title(title)
  fig.savefig(save_path + save_label)